In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import requests
import spotipy
import sys
import pprint
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
from collections import Counter

In [5]:
df_nominee_list = pd.read_csv("Grammy_Nominee.csv")
df_nominee_list.head()

,Title
0,24K Magic
1,Love So Soft
2,Despacito (Remix)
3,Humble
4,Green Light


In [8]:
nominee_list = df_nominee_list["Title"].tolist()
nominee_list

['24K Magic',
 'Love So Soft',
 'Despacito (Remix)',
 'Humble',
 'Green Light',
 'Redbone',
 'The Story Of O.J.',
 'Stay',
 'Million Reasons',
 'Thunder',
 'Feel It Still',
 'Something Just Like This',
 'What About Us',
 '1-800-273-8255',
 'Issues',
 'Praying',
 'Broken Halos',
 'Better Man',
 'Craving You',
 'You Look Good',
 'All the Pretty Girls',
 "Don't Hurt Yourself",
 'Stressed Out',
 'Cheap Thrills',
 'Hotline Bling',
 'Hello',
 'Love Yourself',
 'Closer',
 'Peter Pan',
 'My Church',
 'Brace for Impact (Live a Little)',
 'Am I Wrong',
 'Confident',
 'Dangerous Woman',
 'Piece By Piece (Idol Version)',
 '7 Years',
 'Church Bells',
 "Blue Ain't Your Color",
 'Love Can Go to Hell',
 'Die a Happy Man',
 'Vice',
 'Humble and Kind',
 'Uptown Funk',
 'Blank Space',
 "Can't Feel My Face",
 'Thinking Out Loud',
 'Sugar',
 'Ship to Wreck',
 "Don't Wanna Fight",
 'Really Love',
 'Alright',
 'Traveller',
 'Girl Crush',
 'See You Again (from Furious 7)',
 'Lips Are Movin',
 "Should've Been 

In [9]:
client_credentials_manager = SpotifyClientCredentials("9cc473d2e26f41849b4921a2db73af8a","56f41bc448d64af187db063121d6b82f")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#Genius API
base_url = 'https://api.genius.com'
headers = {'Authorization': 'Bearer '+ '7liNvoUiTMLf18YUqr6UvkCUs-aNzu_N-Go-1AyN7IJDotFSAP6gvIFuxvonmxGo'}
search_url = base_url + '/search'

nominee_features=[]

for song in nominee_list:
    try:
        result = sp.search(song,limit=1, offset=0, type='track')
        spotify_track = result["tracks"]['items'][0]["name"]
        artist = result["tracks"]['items'][0]["artists"][0]["name"]
        spotify_id = result["tracks"]['items'][0]["id"]
        spotify_popularity =result["tracks"]['items'][0]["popularity"]
        duration_ms = result["tracks"]['items'][0]["duration_ms"]
        features = sp.audio_features(spotify_id)[0]
        release_date = result["tracks"]["items"][0]["album"]["release_date"]
        release_year = int(result["tracks"]["items"][0]["album"]["release_date"][:4])
        artist_id = result["tracks"]['items'][0]["album"]['artists'][0]['id']
        artist_response = sp.artist(artist_id)
        artist_followers = artist_response["followers"]["total"]
        artist_popularity = artist_response["popularity"]
        artist_image_url_l= artist_response["images"][2]['url']
        artist_image_url_s= artist_response["images"][1]['url']
        album_url =result["tracks"]['items'][0]["album"]['images'][0]['url']
        data = {'q': spotify_track + ' ' + artist}
        #Genius API Call
        response = requests.get(search_url, data=data, headers=headers)
        json =response.json()
        lyric_url = json['response']['hits'][0]['result']['url']
        page = requests.get(lyric_url)
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics = html.find('div', class_='lyrics').get_text()
        #reformate the lyrics
        lyrics=lyrics.replace('\n'," ")
        lyrics=lyrics.replace("\'"," ")
        lyrics=lyrics.replace(","," ")
        lyrice_total_count = len(lyrics.split(" "))
        lyrice_unique_count = len(Counter(lyrics.split()))
        song_features={
            "song_name":song,
            "track":spotify_track,
            "artist":artist,
            "song_id":spotify_id,
            "popularity":spotify_popularity,
            "duration_ms":duration_ms,
            "release_date":release_date,
            "release_year":release_year,
            "features":features,
            "artist_followers":artist_followers,
            "artist_popularity":artist_popularity,
            "artist_image_url_l":artist_image_url_l,
            "artist_image_url_s":artist_image_url_s,
            "album_url":album_url,
            "lyric_total_word_count":lyrice_total_count,             
            "lyric_unique_word_count":lyrice_unique_count,
            "win/loss":1
        }
    
        nominee_features.append(song_features)
    except:
        pass    
    
print(nominee_features)

[{'song_name': '24K Magic', 'track': '24K Magic', 'artist': 'Bruno Mars', 'song_id': '6b8Be6ljOzmkOmFslEb23P', 'popularity': 79, 'duration_ms': 225983, 'release_date': '2016-11-17', 'release_year': 2016, 'features': {'danceability': 0.818, 'energy': 0.803, 'key': 1, 'loudness': -4.282, 'mode': 1, 'speechiness': 0.0797, 'acousticness': 0.034, 'instrumentalness': 0, 'liveness': 0.153, 'valence': 0.632, 'tempo': 106.97, 'type': 'audio_features', 'id': '6b8Be6ljOzmkOmFslEb23P', 'uri': 'spotify:track:6b8Be6ljOzmkOmFslEb23P', 'track_href': 'https://api.spotify.com/v1/tracks/6b8Be6ljOzmkOmFslEb23P', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6b8Be6ljOzmkOmFslEb23P', 'duration_ms': 225983, 'time_signature': 4}, 'artist_followers': 18302204, 'artist_popularity': 90, 'artist_image_url_l': 'https://i.scdn.co/image/5461dcbb3f23e1df62d85c8273fc49cc4edd49d5', 'artist_image_url_s': 'https://i.scdn.co/image/63cad86cb183568085a4e5f0be86aa187cc511e4', 'album_url': 'https://i.scdn.co/imag

In [10]:
# sep = ':::' # string that doesn't appear in column names

# df_nonimee_features = json_normalize(nonimee_features, sep=sep)
# df_nonimee_features.columns = df_nonimee_features.columns.str.split(sep)
# df_nonimee_features.head(20)

df_nonimee_features = pd.DataFrame.from_dict(json_normalize(nominee_features), orient='columns')
df_nonimee_features

,album_url,artist,artist_followers,artist_image_url_l,artist_image_url_s,artist_popularity,duration_ms,features.acousticness,features.analysis_url,features.danceability,...,features.valence,lyric_total_word_count,lyric_unique_word_count,popularity,release_date,release_year,song_id,song_name,track,win/loss
0,https://i.scdn.co/image/bf41f93f8c895eff007b88...,Bruno Mars,18302204,https://i.scdn.co/image/5461dcbb3f23e1df62d85c...,https://i.scdn.co/image/63cad86cb183568085a4e5...,90,225983,0.034000,https://api.spotify.com/v1/audio-analysis/6b8B...,0.818,...,0.632,549,201,79,2016-11-17,2016,6b8Be6ljOzmkOmFslEb23P,24K Magic,24K Magic,1
1,https://i.scdn.co/image/c5590d3d360a2c278d5cab...,Kelly Clarkson,4160154,https://i.scdn.co/image/5e225f06cf4f2c02262c0f...,https://i.scdn.co/image/9c93bbf952dff60df8a71a...,80,172426,0.005300,https://api.spotify.com/v1/audio-analysis/16QN...,0.808,...,0.758,521,127,66,2017-10-27,2017,16QN8cBubEq706vNtPpOME,Love So Soft,Love So Soft,1
2,https://i.scdn.co/image/909be9469ed835c9a8b039...,Luis Fonsi,4358667,https://i.scdn.co/image/e0b5345bcaeb68a89777c0...,https://i.scdn.co/image/69c7b65c3a597dc76fc392...,85,228826,0.228000,https://api.spotify.com/v1/audio-analysis/6rPO...,0.653,...,0.816,663,277,80,2017-04-17,2017,6rPO02ozF3bM7NnOV4h6s2,Despacito (Remix),Despacito - Remix,1
3,https://i.scdn.co/image/4988546859334f9a5a3fa4...,Kendrick Lamar,11310761,https://i.scdn.co/image/b772a78d4cb192268d6f60...,https://i.scdn.co/image/5259c0496329b3f608a1ae...,91,177000,0.000282,https://api.spotify.com/v1/audio-analysis/7KXj...,0.908,...,0.421,827,221,84,2017-04-14,2017,7KXjTSCq5nL1LoYtL7XAwS,Humble,HUMBLE.,1
4,https://i.scdn.co/image/0623e4c3a9a49f50c07b89...,Lorde,3859718,https://i.scdn.co/image/2f22ea3ea1597887d86441...,https://i.scdn.co/image/ca2e7772dcdfd03401cdd0...,79,234652,0.020900,https://api.spotify.com/v1/audio-analysis/6ie2...,0.594,...,0.253,528,117,74,2017-06-16,2017,6ie2Bw3xLj2JcGowOlcMhb,Green Light,Green Light,1
5,https://i.scdn.co/image/49c99505c9dd13fb27b6bc...,Childish Gambino,4096702,https://i.scdn.co/image/5f0bb754710ffd901940dc...,https://i.scdn.co/image/b51d5cb42b166784f4e1e1...,82,326933,0.172000,https://api.spotify.com/v1/audio-analysis/3kxf...,0.745,...,0.607,356,118,81,2016-12-02,2016,3kxfsdsCpFgN412fpnW85Y,Redbone,Redbone,1
6,https://i.scdn.co/image/ba596dc33258b25898d7a5...,Wiz Kid the Weirdo,54,https://i.scdn.co/image/36775825fd97dda3c1ec90...,https://i.scdn.co/image/7f1012e90166e09339d223...,19,208195,0.489000,https://api.spotify.com/v1/audio-analysis/7AOB...,0.599,...,0.639,21679,5414,31,2017-12-22,2017,7AOBguNeBLIQqGwCMx4eht,The Story Of O.J.,Story of O.J,1
7,https://i.scdn.co/image/37aeea14e41e3dc022fce3...,YG,1419529,https://i.scdn.co/image/447746dc00637fe15c2c2e...,https://i.scdn.co/image/3a88e0a8eb87b98b1e9fe6...,83,237240,0.005520,https://api.spotify.com/v1/audio-analysis/0ZNr...,0.745,...,0.112,551,364,85,2018-08-03,2018,0ZNrc4kNeQYD9koZ3KvCsy,Stay,"BIG BANK (feat. 2 Chainz, Big Sean, Nicki Minaj)",1
8,https://i.scdn.co/image/01c84f9161286cc182cdc4...,Lady Gaga,6056659,https://i.scdn.co/image/4fb4cec0ed27f5e933f3ca...,https://i.scdn.co/image/ab1f49b3c51613a517e01e...,92,205280,0.437000,https://api.spotify.com/v1/audio-analysis/7dZ1...,0.665,...,0.134,438,122,80,2016-10-21,2016,7dZ1Odmx9jWIweQSatnRqo,Million Reasons,Million Reasons,1
9,https://i.scdn.co/image/5232b1dcb43188537b7f9f...,Imagine Dragons,14305824,https://i.scdn.co/image/1cdf5ce3cf329ae433bfa7...,https://i.scdn.co/image/ff8d83627cb2a80d4c1fa3...,94,187146,0.006710,https://api.spotify.com/v1/audio-analysis/1zB4...,0.605,...,0.288,231,87,87,2017-06-23,2017,1zB4vmk8tFRmM9UULNzbLB,Thunder,Thunder,1


In [11]:
df_nonimee_features.to_csv("Grammy_Nominee_Song_Features_image_url.csv",index = False)

In [12]:
#Read Billboard 2010 - 2017 Top Song Date(Exclude Grammy Record Winner)
df_billboard_list = pd.read_csv("Non_Grammy_Song_List(2010-2017).csv")
df_billboard_list.head()

,Title
0,All Falls Down
1,Independent Women Part I
2,Bad Things
3,2 On
4,Roses


In [13]:
#get only the list of title column
billboard_list = df_billboard_list["Title"].tolist()
billboard_list
len(billboard_list)

567

In [14]:
billboard_features=[]

for song in billboard_list:
    try:
        result = sp.search(song,limit=1, offset=0, type='track')
        spotify_track = result["tracks"]['items'][0]["name"]
        artist = result["tracks"]['items'][0]["artists"][0]["name"]
        spotify_id = result["tracks"]['items'][0]["id"]
        spotify_popularity =result["tracks"]['items'][0]["popularity"]
        duration_ms = result["tracks"]['items'][0]["duration_ms"]
        features = sp.audio_features(spotify_id)[0]
        release_date = result["tracks"]["items"][0]["album"]["release_date"]
        release_year = int(result["tracks"]["items"][0]["album"]["release_date"][:4])
        artist_id = result["tracks"]['items'][0]["album"]['artists'][0]['id']
        artist_response = sp.artist(artist_id)
        artist_followers = artist_response["followers"]["total"]
        artist_popularity = artist_response["popularity"]
        artist_image_url_l= artist_response["images"][2]['url']
        artist_image_url_s= artist_response["images"][1]['url']
        album_url =result["tracks"]['items'][0]["album"]['images'][0]['url']
        data = {'q': spotify_track + ' ' + artist}
        #Genius API Call
        response = requests.get(search_url, data=data, headers=headers)
        json =response.json()
        lyric_url = json['response']['hits'][0]['result']['url']
        page = requests.get(lyric_url)
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics = html.find('div', class_='lyrics').get_text()
        #reformate the lyrics
        lyrics=lyrics.replace('\n'," ")
        lyrics=lyrics.replace("\'"," ")
        lyrics=lyrics.replace(","," ")
        lyrice_total_count = len(lyrics.split(" "))
        lyrice_unique_count = len(Counter(lyrics.split()))
        song_features={
            "song_name":song,
            "track":spotify_track,
            "artist":artist,
            "song_id":spotify_id,
            "popularity":spotify_popularity,
            "duration_ms":duration_ms,
            "release_date":release_date,
            "release_year":release_year,
            "features":features,
            "artist_followers":artist_followers,
            "artist_popularity":artist_popularity,
            "artist_image_url_l":artist_image_url_l,
            "artist_image_url_s":artist_image_url_s,
            "album_url":album_url,
            "lyric_total_word_count":lyrice_total_count,             
            "lyric_unique_word_count":lyrice_unique_count,
            "win/loss":1
        }
    
        billboard_features.append(song_features)
    except:
        pass    
    
print(billboard_features)

[{'song_name': 'All Falls Down', 'track': 'All Falls Down (feat. Juliander)', 'artist': 'Alan Walker', 'song_id': '5yFSF6qQA1rcLsQRnBsZgb', 'popularity': 80, 'duration_ms': 199111, 'release_date': '2017-10-27', 'release_year': 2017, 'features': {'danceability': 0.658, 'energy': 0.678, 'key': 1, 'loudness': -4.407, 'mode': 1, 'speechiness': 0.058, 'acousticness': 0.303, 'instrumentalness': 0, 'liveness': 0.361, 'valence': 0.662, 'tempo': 97.976, 'type': 'audio_features', 'id': '5yFSF6qQA1rcLsQRnBsZgb', 'uri': 'spotify:track:5yFSF6qQA1rcLsQRnBsZgb', 'track_href': 'https://api.spotify.com/v1/tracks/5yFSF6qQA1rcLsQRnBsZgb', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5yFSF6qQA1rcLsQRnBsZgb', 'duration_ms': 199111, 'time_signature': 4}, 'artist_followers': 5400694, 'artist_popularity': 86, 'artist_image_url_l': 'https://i.scdn.co/image/9221df357035db0fefe1bda40a8e2c3377bfaad9', 'artist_image_url_s': 'https://i.scdn.co/image/55b19e031041baad492dbf49a920a424fb5721a8', 'album_ur

In [15]:
df_billboard_features = pd.DataFrame.from_dict(json_normalize(billboard_features), orient='columns')
df_billboard_features

,album_url,artist,artist_followers,artist_image_url_l,artist_image_url_s,artist_popularity,duration_ms,features.acousticness,features.analysis_url,features.danceability,...,features.valence,lyric_total_word_count,lyric_unique_word_count,popularity,release_date,release_year,song_id,song_name,track,win/loss
0,https://i.scdn.co/image/802f9b8d3391cc8d9e0aa3...,Alan Walker,5400694,https://i.scdn.co/image/9221df357035db0fefe1bd...,https://i.scdn.co/image/55b19e031041baad492dbf...,86,199111,0.303000,https://api.spotify.com/v1/audio-analysis/5yFS...,0.658,...,0.662,29004,11798,80,2017-10-27,2017,5yFSF6qQA1rcLsQRnBsZgb,All Falls Down,All Falls Down (feat. Juliander),1
1,https://i.scdn.co/image/241aa0296152106b19e6d3...,Graham BLVD,411751,https://i.scdn.co/image/853d77fb4c4834e2b93080...,https://i.scdn.co/image/f30641e62cc7ff58f96c68...,0,218363,0.476000,https://api.spotify.com/v1/audio-analysis/0KoM...,0.807,...,0.815,9093,2775,0,2017-11-09,2017,0KoMYH9cA7RHuVMrwYPchn,Independent Women Part I,Independent Women Part I,1
2,https://i.scdn.co/image/bccf49605f8146e3133015...,Machine Gun Kelly,1288988,https://i.scdn.co/image/67f0e4ca9a5dba6b8497c8...,https://i.scdn.co/image/bb5281a25dac235b863259...,79,239293,0.214000,https://api.spotify.com/v1/audio-analysis/1PSB...,0.697,...,0.305,3020,1279,73,2017-05-12,2017,1PSBzsahR2AKwLJgx8ehBj,Bad Things,Bad Things (with Camila Cabello),1
3,https://i.scdn.co/image/9773c3e75a9f9658f27820...,Yella Beezy,98158,https://i.scdn.co/image/a4822a02c6641691a565dc...,https://i.scdn.co/image/3c5a60f6841520c5bd56fc...,69,219552,0.028800,https://api.spotify.com/v1/audio-analysis/5c0V...,0.803,...,0.538,1095,218,73,2017-12-21,2017,5c0Vl3yklw5WX9YI36dPLd,2 On,That's On Me,1
4,https://i.scdn.co/image/14cfe6670c2d0b871f098e...,Lil Skies,1771497,https://i.scdn.co/image/1ca2a6d7a0f3e7bd8ccf89...,https://i.scdn.co/image/e2d587b94e8375f4ba57c8...,85,262544,0.547000,https://api.spotify.com/v1/audio-analysis/3ZhT...,0.691,...,0.456,819,281,80,2017-12-12,2017,3ZhTT6yjZwpPph5MIJ53XY,Roses,Red Roses (feat. Landon Cube),1
5,https://i.scdn.co/image/b6ff79f9faeab5753a5d6e...,Big Sean,5322100,https://i.scdn.co/image/75050184de713540aab408...,https://i.scdn.co/image/3e078572245ba8875cc2f0...,85,222360,0.105000,https://api.spotify.com/v1/audio-analysis/0SGk...,0.770,...,0.260,801,300,77,2017-02-03,2017,0SGkqnVQo9KPytSri1H6cF,Bounce Back,Bounce Back,1
6,https://i.scdn.co/image/8984042f24e62f3e53ebfa...,Trippie Redd,2204059,https://i.scdn.co/image/fcdf7e2a361b05c614f108...,https://i.scdn.co/image/faf084ae74f1113128fa23...,92,129429,0.136000,https://api.spotify.com/v1/audio-analysis/0Aci...,0.544,...,0.117,222,76,79,2018-11-09,2018,0AcicReEGm1qcENjqzh1Tg,Scars,Love Scars 3,1
7,https://i.scdn.co/image/e8468776f7a0e12af7e9ad...,Ayo & Teo,541432,https://i.scdn.co/image/83dc4d7f45e4ec2d76ff75...,https://i.scdn.co/image/bd8ae4a7f7c1611ea80b94...,67,238586,0.083700,https://api.spotify.com/v1/audio-analysis/0pSB...,0.804,...,0.789,730,152,75,2017-03-15,2017,0pSBuHjILhNEo55xK1zrRt,Rolex,Rolex,1
8,https://i.scdn.co/image/32dbd227fc8cc94fb7ec9a...,Hugh Jackman,411751,https://i.scdn.co/image/853d77fb4c4834e2b93080...,https://i.scdn.co/image/f30641e62cc7ff58f96c68...,0,302146,0.000239,https://api.spotify.com/v1/audio-analysis/4ylW...,0.417,...,0.400,726,174,80,2017-12-08,2017,4ylWMuGbMXNDgDd8lErEle,The Greatest,The Greatest Show,1
9,https://i.scdn.co/image/f56f23cbd3783f8594a487...,Migos,7841339,https://i.scdn.co/image/98b8cc9b24841b0f516dfc...,https://i.scdn.co/image/f31fe04e6bfd7d154cc643...,87,242407,0.242000,https://api.spotify.com/v1/audio-analysis/7KOl...,0.865,...,0.486,722,283,74,2017-01-27,2017,7KOlJ92bu51cltsD9KU5I7,T-Shirt,T-Shirt,1


In [16]:
df_billboard_features.to_csv("Billboard_Non_Grammy_Song_Features(2010-2017).csv",index = False)

In [18]:
#Combin grammy and billboard song data
full_song_list = pd.concat([df_billboard_features, df_nonimee_features], ignore_index=True)
full_song_list.head()

,album_url,artist,artist_followers,artist_image_url_l,artist_image_url_s,artist_popularity,duration_ms,features.acousticness,features.analysis_url,features.danceability,...,features.valence,lyric_total_word_count,lyric_unique_word_count,popularity,release_date,release_year,song_id,song_name,track,win/loss
0,https://i.scdn.co/image/802f9b8d3391cc8d9e0aa3...,Alan Walker,5400694,https://i.scdn.co/image/9221df357035db0fefe1bd...,https://i.scdn.co/image/55b19e031041baad492dbf...,86,199111,0.3030,https://api.spotify.com/v1/audio-analysis/5yFS...,0.658,...,0.662,29004,11798,80,2017-10-27,2017,5yFSF6qQA1rcLsQRnBsZgb,All Falls Down,All Falls Down (feat. Juliander),1
1,https://i.scdn.co/image/241aa0296152106b19e6d3...,Graham BLVD,411751,https://i.scdn.co/image/853d77fb4c4834e2b93080...,https://i.scdn.co/image/f30641e62cc7ff58f96c68...,0,218363,0.4760,https://api.spotify.com/v1/audio-analysis/0KoM...,0.807,...,0.815,9093,2775,0,2017-11-09,2017,0KoMYH9cA7RHuVMrwYPchn,Independent Women Part I,Independent Women Part I,1
2,https://i.scdn.co/image/bccf49605f8146e3133015...,Machine Gun Kelly,1288988,https://i.scdn.co/image/67f0e4ca9a5dba6b8497c8...,https://i.scdn.co/image/bb5281a25dac235b863259...,79,239293,0.2140,https://api.spotify.com/v1/audio-analysis/1PSB...,0.697,...,0.305,3020,1279,73,2017-05-12,2017,1PSBzsahR2AKwLJgx8ehBj,Bad Things,Bad Things (with Camila Cabello),1
3,https://i.scdn.co/image/9773c3e75a9f9658f27820...,Yella Beezy,98158,https://i.scdn.co/image/a4822a02c6641691a565dc...,https://i.scdn.co/image/3c5a60f6841520c5bd56fc...,69,219552,0.0288,https://api.spotify.com/v1/audio-analysis/5c0V...,0.803,...,0.538,1095,218,73,2017-12-21,2017,5c0Vl3yklw5WX9YI36dPLd,2 On,That's On Me,1
4,https://i.scdn.co/image/14cfe6670c2d0b871f098e...,Lil Skies,1771497,https://i.scdn.co/image/1ca2a6d7a0f3e7bd8ccf89...,https://i.scdn.co/image/e2d587b94e8375f4ba57c8...,85,262544,0.5470,https://api.spotify.com/v1/audio-analysis/3ZhT...,0.691,...,0.456,819,281,80,2017-12-12,2017,3ZhTT6yjZwpPph5MIJ53XY,Roses,Red Roses (feat. Landon Cube),1


In [19]:
full_song_list.to_csv("Nomination_ML_url_2010_2017.CSV",index = False)

In [20]:
#get song features for 2018 top song(for Predict)
#source:https://www.officialcharts.com/chart-news/the-official-top-40-biggest-songs-of-2018-so-far__22432/
#https://en.wikipedia.org/wiki/List_of_Billboard_Hot_100_top-ten_singles_in_2018
df_2018_list = pd.read_csv("2018_top_song_list.csv")
df_2018_list.head()

,artist,Title
0,Camila Cabello,Havana
1,G-Eazy,No Limit
2,Halsey,Bad at Love
3,Panic! At The Disco,High Hopes
4,Ariana Grande,"Thank U, Next"


In [21]:
song2018_list = df_2018_list["Title"].tolist()
song2018_list

['Havana',
 'No Limit',
 'Bad at Love',
 'High Hopes',
 'Thank U, Next',
 'Without Me',
 'Mo Bamba',
 'Zeze',
 'Mia',
 'Drip Too Hard',
 'Shallow',
 'Happier',
 'Mona Lisa',
 "Don't Cry",
 'Uproar',
 'Let It Fly',
 'Killshot',
 'Youngblood',
 'I Love It',
 'Lucky You',
 'The Ringer',
 'Love Lies',
 'God Is a Woman',
 'Sicko Mode',
 'Stargazing',
 'No Brainer',
 'Fefe',
 'Taste',
 'Nonstop',
 'In My Feelings',
 "I'm Upset",
 'Emotionless',
 "Don't Matter to Me",
 'Girls Like You',
 'Yikes',
 'Lucid Dreams',
 'Yes Indeed',
 "Boo'd Up",
 'Fake Love',
 'This Is America',
 'Never Be the Same',
 'Better Now',
 'No Tears Left to Cry',
 'ATM',
 "Kevin's Heart",
 'KOD',
 'Chun-Li',
 'Nice for What',
 'I Like It',
 'Call Out My Name',
 'Walk It Talk It',
 'Sad!',
 'Freaky Friday',
 'The Middle',
 'Psycho',
 'Look Alive',
 'All the Stars',
 'Pray for Me',
 'Stir Fry',
 'Meant to Be',
 "God's Plan",
 'Diplomatic Immunity',
 'Finesse',
 'Filthy',
 'Bodak Yellow',
 'Too Good at Goodbyes',
 'Rockstar

In [22]:
song2018_features=[]

for song in song2018_list:
    try:
        result = sp.search(song,limit=1, offset=0, type='track')
        spotify_track = result["tracks"]['items'][0]["name"]
        artist = result["tracks"]['items'][0]["artists"][0]["name"]
        spotify_id = result["tracks"]['items'][0]["id"]
        spotify_popularity =result["tracks"]['items'][0]["popularity"]
        duration_ms = result["tracks"]['items'][0]["duration_ms"]
        features = sp.audio_features(spotify_id)[0]
        release_date = result["tracks"]["items"][0]["album"]["release_date"]
        release_year = int(result["tracks"]["items"][0]["album"]["release_date"][:4])
        artist_id = result["tracks"]['items'][0]["album"]['artists'][0]['id']
        artist_response = sp.artist(artist_id)
        artist_followers = artist_response["followers"]["total"]
        artist_popularity = artist_response["popularity"]
        artist_image_url_l= artist_response["images"][2]['url']
        artist_image_url_s= artist_response["images"][1]['url']
        album_url =result["tracks"]['items'][0]["album"]['images'][0]['url']
        data = {'q': spotify_track + ' ' + artist}
        #Genius API Call
        response = requests.get(search_url, data=data, headers=headers)
        json =response.json()
        lyric_url = json['response']['hits'][0]['result']['url']
        page = requests.get(lyric_url)
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics = html.find('div', class_='lyrics').get_text()
        #reformate the lyrics
        lyrics=lyrics.replace('\n'," ")
        lyrics=lyrics.replace("\'"," ")
        lyrics=lyrics.replace(","," ")
        lyrice_total_count = len(lyrics.split(" "))
        lyrice_unique_count = len(Counter(lyrics.split()))
        song_features={
            "song_name":song,
            "track":spotify_track,
            "artist":artist,
            "song_id":spotify_id,
            "popularity":spotify_popularity,
            "duration_ms":duration_ms,
            "release_date":release_date,
            "release_year":release_year,
            "features":features,
            "artist_followers":artist_followers,
            "artist_popularity":artist_popularity,
            "artist_image_url_l":artist_image_url_l,
            "artist_image_url_s":artist_image_url_s,
            "album_url":album_url,
            "lyric_total_word_count":lyrice_total_count,             
            "lyric_unique_word_count":lyrice_unique_count,
            "win/loss":1
        }
    
        song2018_features.append(song_features)
    except:
        pass    
    
print(song2018_features)

[{'song_name': 'Havana', 'track': 'Havana', 'artist': 'Camila Cabello', 'song_id': '1rfofaqEpACxVEHIZBJe6W', 'popularity': 87, 'duration_ms': 217306, 'release_date': '2018-01-12', 'release_year': 2018, 'features': {'danceability': 0.765, 'energy': 0.523, 'key': 2, 'loudness': -4.333, 'mode': 1, 'speechiness': 0.03, 'acousticness': 0.184, 'instrumentalness': 3.56e-05, 'liveness': 0.132, 'valence': 0.394, 'tempo': 104.988, 'type': 'audio_features', 'id': '1rfofaqEpACxVEHIZBJe6W', 'uri': 'spotify:track:1rfofaqEpACxVEHIZBJe6W', 'track_href': 'https://api.spotify.com/v1/tracks/1rfofaqEpACxVEHIZBJe6W', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1rfofaqEpACxVEHIZBJe6W', 'duration_ms': 217307, 'time_signature': 4}, 'artist_followers': 7398818, 'artist_popularity': 88, 'artist_image_url_l': 'https://i.scdn.co/image/aa9f03ea0bd3bda2683a6598361942561f7fd3aa', 'artist_image_url_s': 'https://i.scdn.co/image/7dce1aa42eed23fe5bba6085babc98ac760b3947', 'album_url': 'https://i.scdn.co/i

In [23]:
df_song2018 = pd.DataFrame.from_dict(json_normalize(song2018_features), orient='columns')
df_song2018

,album_url,artist,artist_followers,artist_image_url_l,artist_image_url_s,artist_popularity,duration_ms,features.acousticness,features.analysis_url,features.danceability,...,features.valence,lyric_total_word_count,lyric_unique_word_count,popularity,release_date,release_year,song_id,song_name,track,win/loss
0,https://i.scdn.co/image/8ebf0216fa9d294177e79c...,Camila Cabello,7398818,https://i.scdn.co/image/aa9f03ea0bd3bda2683a65...,https://i.scdn.co/image/7dce1aa42eed23fe5bba60...,88,217306,0.18400,https://api.spotify.com/v1/audio-analysis/1rfo...,0.765,...,0.3940,626,180,87,2018-01-12,2018,1rfofaqEpACxVEHIZBJe6W,Havana,Havana,1
1,https://i.scdn.co/image/ad39af99931e17c24648e8...,G-Eazy,3563212,https://i.scdn.co/image/b1b5076582c4a9eb39a4bb...,https://i.scdn.co/image/9c35fbd7d6cf90c1f31e3a...,86,245386,0.01170,https://api.spotify.com/v1/audio-analysis/2Xqd...,0.838,...,0.4050,991,252,77,2017-09-07,2017,2Xqd0wUttjueBfdcltADOv,No Limit,No Limit,1
2,https://i.scdn.co/image/c8af4aa7e5b7a022d9cb49...,Halsey,4393493,https://i.scdn.co/image/f0d471f9ae4bbb4d9e970b...,https://i.scdn.co/image/0775d3134d1d2469591dfe...,91,181279,0.06040,https://api.spotify.com/v1/audio-analysis/7y9i...,0.675,...,0.6120,524,150,79,2017-06-02,2017,7y9iMe8SOB6z3NoHE2OfXl,Bad at Love,Bad At Love,1
3,https://i.scdn.co/image/5a0c73915586db4a6acf0a...,Panic! At The Disco,5418730,https://i.scdn.co/image/5b0544b8a3898be8693915...,https://i.scdn.co/image/e71f9ba6573c95041ecd71...,88,190946,0.19300,https://api.spotify.com/v1/audio-analysis/1rqq...,0.579,...,0.6810,544,124,90,2018-06-22,2018,1rqqCSm0Qe4I9rUvWncaom,High Hopes,High Hopes,1
4,https://i.scdn.co/image/37bbb4711235a0128db66a...,Ariana Grande,20841651,https://i.scdn.co/image/a1063f43437d1d724a3d7d...,https://i.scdn.co/image/6c60edb78bf9b1c0eeb219...,97,207333,0.28000,https://api.spotify.com/v1/audio-analysis/2rPE...,0.724,...,0.4350,586,175,100,2018-11-03,2018,2rPE9A1vEgShuZxxzR2tZH,"Thank U, Next","thank u, next",1
5,https://i.scdn.co/image/dd916d01780a9591c8a029...,Halsey,4393495,https://i.scdn.co/image/f0d471f9ae4bbb4d9e970b...,https://i.scdn.co/image/0775d3134d1d2469591dfe...,91,201660,0.29700,https://api.spotify.com/v1/audio-analysis/5p7u...,0.752,...,0.5330,510,142,96,2018-10-04,2018,5p7ujcrUXASCNwRaWNHR1C,Without Me,Without Me,1
6,https://i.scdn.co/image/0e3e187e3ae399c7aa68c6...,Sheck Wes,190935,https://i.scdn.co/image/46d53d0c034b4ba25764b4...,https://i.scdn.co/image/12019a4e186428cc30ab7f...,83,183906,0.19400,https://api.spotify.com/v1/audio-analysis/1xzB...,0.729,...,0.2610,417,166,93,2018-10-05,2018,1xzBco0xcoJEDXktl7Jxrr,Mo Bamba,Mo Bamba,1
7,https://i.scdn.co/image/7f8b5350c0f6b6fd754fb5...,Kodak Black,3899719,https://i.scdn.co/image/5a428596636f28fb55b5bb...,https://i.scdn.co/image/ef90a19321978c44312468...,91,228653,0.07100,https://api.spotify.com/v1/audio-analysis/7l3E...,0.826,...,0.5430,613,412,95,2018-10-12,2018,7l3E7lcozEodtVsSTCkcaA,Zeze,ZEZE (feat. Travis Scott & Offset),1
8,https://i.scdn.co/image/80c0f2b0ad6669f3abd91c...,Bad Bunny,9530820,https://i.scdn.co/image/96532e96a03953bee4fb79...,https://i.scdn.co/image/b0c68f05a4df3f9c362238...,95,210367,0.01410,https://api.spotify.com/v1/audio-analysis/116H...,0.817,...,0.1580,613,412,97,2018-10-11,2018,116H0KvKr2Zl4RPuVBruDO,Mia,MIA (feat. Drake),1
9,https://i.scdn.co/image/abd55a98fc8e9c45bacac1...,Lil Baby,1029173,https://i.scdn.co/image/5190f528f0c4a9ce381594...,https://i.scdn.co/image/2b08bf52411ab9a644e18c...,91,145603,0.09450,https://api.spotify.com/v1/audio-analysis/1Bxk...,0.900,...,0.3990,657,192,90,2018-09-12,2018,1BxkZE73h9BN3qwuA15TA3,Drip Too Hard,Drip Too Hard,1


In [24]:
df_song2018.to_csv("2018_top_song_features_images_url.CSV",index = False)